In [18]:
import sys

import numpy as np
import pandas as pd
import s3fs

sys.path.append("/Users/fgu/dev/projects/mdb_eval")
import src.helpers.io as io
import src.helpers.data as hd
import src.data.aggregators as agg
import src.data.selectors as sl
import src.data.make_data as md

fs = s3fs.S3FileSystem(profile="3di")

pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data

In [ ]:
fp = 's3://3di-data-mdb/clean'
# fp = 's3://3di-project-eval/'
fs.ls(fp, refresh=True, detail=False)

['3di-data-mdb/clean/',
 '3di-data-mdb/clean/mdb_000.parquet',
 '3di-data-mdb/clean/mdb_111.parquet',
 '3di-data-mdb/clean/mdb_777.parquet',
 '3di-data-mdb/clean/mdb_X11.parquet',
 '3di-data-mdb/clean/mdb_X77.parquet',
 '3di-data-mdb/clean/mdb_costa.parquet',
 '3di-data-mdb/clean/pieces',
 '3di-data-mdb/clean/samples']

In [55]:
dfa = io.read_parquet('s3://3di-project-eval/eval.parquet')
hd.inspect(dfa)

shape: (166,249, 28), users: 5884


,user_id,ym,ymn,txns_count,txns_volume,month_income,inflows,outflows,netflows,netflows_norm,inflows_norm,outflows_norm,has_pos_netflows,pos_netflows,t,tt,month_spend,user_registration_date,age,is_female,region,is_urban,has_sa_account,generation,generation_code,new_loan,prop_credit,disc_spend
0,361990,2016-04,201604,4767.620117,4767.620117,4947.571289,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,-6,1002.900024,2016-10-31,33.0,0.0,south east,0.0,True,Millennials,3,0,0.0,0.107366
1,361990,2016-05,201605,4787.720215,4787.720215,4947.571289,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,-5,1881.459961,2016-10-31,33.0,0.0,south east,0.0,True,Millennials,3,0,0.0,0.219724


In [7]:
dft = md.read_piece('~/tmp/mdb_111.parquet')
hd.inspect(dft)

shape: (166,249, 27), users: 5884


,user_id,ym,ymn,txns_count,txns_volume,month_income,inflows,outflows,netflows,netflows_norm,inflows_norm,outflows_norm,has_pos_netflows,pos_netflows_norm,t,tt,month_spend,user_registration_date,age,is_female,region,is_urban,has_sa_account,generation,generation_code,new_loan,pct_credit
0,361990,2016-04,201604,80,4767.620117,4947.571289,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,-6,1002.900024,2016-10-31,33.0,0.0,south east,0.0,True,Millennials,3,0,0.0
1,361990,2016-05,201605,156,4787.720215,4947.571289,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0,-5,1881.459961,2016-10-31,33.0,0.0,south east,0.0,True,Millennials,3,0,0.0


Reading ~/tmp/mdb_111.parquet
Time for read_piece                    : 1.18 minutes
shape: (593,133, 35), users: 270


,date,user_id,amount,desc,merchant,tag_group,tag_spend,user_registration_date,account_created,account_id,account_last_refreshed,account_provider,account_type,birth_year,data_warehouse_date_created,data_warehouse_date_last_updated,id,is_debit,is_female,is_sa_flow,is_salary_pmt,is_urban,latest_balance,lsoa,merchant_business_line,msoa,postcode,region_name,salary_range,tag,tag_auto,tag_manual,tag_up,updated_flag,ym
0,2012-01-03,111,-114.0,<mdbremoved>,NaN,NaN,NaN,2010-06-19,2016-10-30,235634,2018-08-08 10:24:00,lloyds bank,current,NaN,2014-07-18,2015-03-19,397323,False,0.0,False,False,1.0,0.5,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01
1,2012-01-03,111,8.5,<mdbremoved>,NaN,NaN,NaN,2010-06-19,2016-10-30,235634,2018-08-08 10:24:00,lloyds bank,current,NaN,2014-07-18,2015-03-19,397324,True,0.0,False,False,1.0,0.5,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01


## dev

In [56]:
dfa.describe().round(2).T

,count,mean,std,min,25%,50%,75%,max
user_id,166249.0,478278.95,59842.84,338943.00,428153.00,476204.00,527240.00,589752.00
ymn,166249.0,201830.01,98.83,201508.00,201801.00,201810.00,201906.00,202008.00
txns_count,166249.0,17086.55,17026.22,207.40,6080.88,10764.37,20400.93,68808.30
txns_volume,166249.0,22957.81,76135.95,207.40,6080.88,10764.37,20400.93,9275361.00
month_income,166249.0,2923.28,1945.61,417.25,1531.95,2308.12,3756.86,8047.88
inflows,166249.0,504.71,1020.76,0.00,0.00,0.00,450.00,4000.00
outflows,166249.0,530.81,1109.19,0.00,0.00,0.00,441.54,4269.60
netflows,166249.0,-26.01,1187.77,-4500.00,0.00,0.00,70.00,3900.00
netflows_norm,166249.0,-0.01,0.44,-1.65,0.00,0.00,0.03,1.40
inflows_norm,166249.0,0.20,0.39,0.00,0.00,0.00,0.19,1.50


## dev -- agg tester

In [9]:
dft

,date,user_id,amount,desc,merchant,tag_group,tag_spend,user_registration_date,account_created,account_id,account_last_refreshed,account_provider,account_type,birth_year,data_warehouse_date_created,data_warehouse_date_last_updated,id,is_debit,is_female,is_sa_flow,is_salary_pmt,is_urban,latest_balance,lsoa,merchant_business_line,msoa,postcode,region_name,salary_range,tag,tag_auto,tag_manual,tag_up,updated_flag,ym
0,2012-01-03,111,-114.00,<mdbremoved>,NaN,NaN,NaN,2010-06-19,2016-10-30,235634,2018-08-08 10:24:00,lloyds bank,current,NaN,2014-07-18,2015-03-19,397323,False,0.0,False,False,1.0,0.500000,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01
1,2012-01-03,111,8.50,<mdbremoved>,NaN,NaN,NaN,2010-06-19,2016-10-30,235634,2018-08-08 10:24:00,lloyds bank,current,NaN,2014-07-18,2015-03-19,397324,True,0.0,False,False,1.0,0.500000,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01
2,2012-01-03,111,-290.00,<mdbremoved>,NaN,NaN,NaN,2010-06-19,2016-10-30,235638,2018-08-08 10:24:00,lloyds bank,savings,NaN,2014-07-18,2015-03-19,397457,False,0.0,True,False,1.0,NaN,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01
3,2012-01-03,111,-0.97,save the change,NaN,transfers,NaN,2010-06-19,2016-10-30,235633,2017-10-29 11:19:00,lloyds bank,current,NaN,2014-07-18,2017-08-15,397122,False,0.0,False,False,1.0,NaN,e01001036,personal,e02000222,cr2 7,london,,other_transfers,transfers,no tag,no tag,u,2012-01
4,2012-01-03,111,4.12,6124.2310.5186 all pl 21.00 xr 5.09709 cd 4729,NaN,NaN,NaN,2010-06-19,2016-10-30,235634,2018-08-08 10:24:00,lloyds bank,current,NaN,2014-07-18,2017-10-23,397321,True,0.0,False,False,1.0,0.500000,e01001036,NaN,e02000222,cr2 7,london,,NaN,NaN,no tag,no tag,u,2012-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
593128,2020-07-29,589111,5.00,ncp parkpass www,ncp,spend,vehicle,2020-07-23,2020-07-23,1725968,2020-08-01 23:13:00,hsbc,current,1999.0,2020-07-31,2020-08-01,802973902,True,0.0,False,False,1.0,246.880005,e01017092,ncp,e02003530,po2 9,south east,30k to 40k,motor,parking or tolls,no tag,parking,u,2020-07
593129,2020-07-29,589111,2.80,tesco store 3048 portsmouth,tesco,spend,groceries,2020-07-23,2020-07-23,1725968,2020-08-01 23:13:00,hsbc,current,1999.0,2020-07-31,1900-01-01,802973901,True,0.0,False,False,1.0,246.880005,e01017092,tesco supermarket,e02003530,po2 9,south east,30k to 40k,household,"food, groceries, household",no tag,"food, groceries, household",c,2020-07
593130,2020-07-30,589111,7.49,just eat london,just eat,spend,eating out,2020-07-23,2020-07-23,1725968,2020-08-01 23:13:00,hsbc,current,1999.0,2020-07-31,1900-01-01,802973900,True,0.0,False,False,1.0,246.880005,e01017092,just eat,e02003530,po2 9,south east,30k to 40k,services,take-away,no tag,take-away,c,2020-07
593131,2020-07-31,589111,0.59,home bargains portsmouth,home bargains,spend,groceries,2020-07-23,2020-07-23,1725968,2020-08-01 23:13:00,hsbc,current,1999.0,2020-08-02,1900-01-01,803502331,True,0.0,False,False,1.0,246.880005,e01017092,home bargains,e02003530,po2 9,south east,30k to 40k,household,"food, groceries, household",no tag,"food, groceries, household",c,2020-07


In [11]:
dft.merchant.isin(['tesco'])

0         False
1         False
2         False
3         False
4         False
          ...  
593128    False
593129     True
593130    False
593131    False
593132    False
Name: merchant, Length: 593133, dtype: bool

In [50]:
k = agg.discretionary_spend(dft)
k

user_id  ym     
111      2012-01    0.031087
         2012-02    0.000000
         2012-03    0.056978
         2012-04    0.154500
         2012-05    0.090894
                      ...   
589111   2020-03    0.998623
         2020-04    0.459781
         2020-05    0.209073
         2020-06    0.226909
         2020-07    0.219720
Name: disc_spend, Length: 7303, dtype: float64

In [51]:
k.describe()

count      7303.000000
mean        317.039017
std        3391.583805
min           0.000000
25%           0.086906
50%           0.325490
75%           1.377958
max      117432.007812
Name: disc_spend, dtype: float64